## MapReduceRetrieverWithReferences

### Overview
A ideia de criar esta classe é para ajudar em implementações complexas onde precisamos processar e combinar documentos de forma eficiente, utilizando o conceito de **Map-Reduce**. Isso é particularmente útil em cenários de **busca de documentos** e **resumo** quando há um grande volume de dados ou documentos a serem processados. Além disso, ela oferece a capacidade de incluir referências e metadados dos documentos processados, o que facilita a rastreabilidade e a apresentação dos resultados de maneira mais informativa.

### Objetivo
A classe **`MapReduceRetrieverWithReferences`** é projetada para:
- **Recuperar documentos** relevantes de uma fonte (como um buscador ou retriever).
- **Processar documentos** em paralelo, gerando resumos individuais (Map Step).
- **Combinar os resumos** gerados em uma resposta coesa (Reduce Step).
- **Incluir metadados e referências** dos documentos processados, como IDs e URLs.

1. **Map Step**:
   - O **LLM** gera **um resumo por documento** com base no **map_prompt_template**.
   - Cada documento é processado separadamente.

2. **Reduce Step**:
   - O **LLM** combina os **resumos gerados** no **map step**.
   - Ele usa o **reduce_prompt_template** para criar um **resumo final** que é coeso e responde à pergunta do usuário com base em todos os documentos processados.

#### Exemplo:

Imagine que a pergunta do usuário seja "Qual é a capital do Japão?", e você tenha três documentos sobre o Japão:

- **Documento 1**: Fala sobre a geografia do Japão.
- **Documento 2**: Fala sobre a história do Japão.
- **Documento 3**: Fala sobre Tóquio como capital do Japão.

##### Passo 1: Map Step
- **Resumo do Documento 1**: "O Japão é um arquipélago localizado no leste da Ásia."
- **Resumo do Documento 2**: "O Japão tem uma rica história que inclui o período Edo."
- **Resumo do Documento 3**: "Tóquio é a capital do Japão."

##### Passo 2: Reduce Step
- **Resumo Final (Reduce)**: "O Japão é um arquipélago no leste da Ásia, com uma rica história. Tóquio é sua capital."

#### Resumo:

- **Map Step**: Gera **um resumo por documento**.
- **Reduce Step**: Gera um **resumo final** que combina os resumos anteriores de maneira coesa e focada.


## **Implementação da classe MapReduceRetriever**

### Estrutura da Classe

**`__init__(self, llm, retriever, document_prompt_template, map_prompt_template, reduce_prompt_template)`**  
Este método inicializa a classe e define os componentes necessários:
- `llm`: Modelo de linguagem utilizado para processamento dos documentos.
- `retriever`: Responsável por buscar documentos relevantes com base na consulta do usuário.
- `document_prompt_template`: Template usado para formatar os documentos.
- `map_prompt_template`: Template usado para resumir cada documento individualmente.
- `reduce_prompt_template`: Template usado para combinar os resumos de forma coesa.  

**`_initialize_map_reduce_chain(self)`**
Este método configura a cadeia **Map-Reduce** usando os templates fornecidos. Ele define:
- **LLMChain para Map Step**: Utilizado para resumir cada documento individualmente.
- **LLMChain para Reduce Step**: Usado para combinar os resumos gerados no Map Step.
- **StuffDocumentsChain**: Cadeia responsável por formatar e combinar os documentos no Map Step.
- **ReduceDocumentsChain**: Cadeia que combina os documentos após o Map Step.

```python
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.combine_documents.map_reduce import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chains.llm import LLMChain


class MapReduceRetrieverWithReferences:
    def __init__(self, llm, retriever, document_prompt_template, map_prompt_template, reduce_prompt_template):
        self.llm = llm
        self.retriever = retriever
        self.document_prompt_template = document_prompt_template
        self.map_prompt_template = map_prompt_template
        self.reduce_prompt_template = reduce_prompt_template
        self.map_reduce_chain = self._initialize_map_reduce_chain()


    def _initialize_map_reduce_chain(self):    
        # Cadeia LLM para processar cada documento individualmente (Map Step)
        llm_chain_map = LLMChain(llm=self.llm, prompt=self.map_prompt_template)

        # Cadeia LLM para combinar os resumos gerados (Reduce Step)
        llm_chain_reduce = LLMChain(llm=self.llm, prompt=self.reduce_prompt_template)

        # Cadeia para formatar e combinar documentos no Map Step
        combine_documents_chain = StuffDocumentsChain(
            llm_chain=llm_chain_map,
            document_prompt=self.document_prompt_template,
            document_variable_name="context"
        )

        # Cadeia de redução para combinar os documentos após o map step
        reduce_documents_chain = ReduceDocumentsChain(
            combine_documents_chain=combine_documents_chain,
            llm_chain=llm_chain_reduce  # Usando o reduce_prompt_template aqui
        )

        # Cadeia Map-Reduce para processar os documentos e gerar as respostas
        map_reduce_chain = MapReduceDocumentsChain(
            llm_chain=llm_chain_map,  # Cadeia Map
            reduce_documents_chain=reduce_documents_chain  # Cadeia Reduce
        )
        
        return map_reduce_chain

    def retrieve_and_combine(self, user_input: str):
        # Recupera os documentos relevantes
        docs = self.retriever.get_relevant_documents(user_input)

        # Chama a cadeia Map-Reduce com os documentos
        result, doc_references = self.map_reduce_chain.invoke({"context": docs})

        # Retorna o resultado final e as referências dos documentos
        return result, doc_references

    def format_response_with_references(self, result, docs):
        # Formatar a resposta com as referências
        doc_references = []
        for doc in docs:
            doc_id = doc.metadata.get('doc_id', 'Unknown')
            title = doc.metadata.get('title', 'Sem título')
            url = doc.metadata.get('url', 'Unknown')
            doc_references.append({
                "doc_id": doc_id,
                "title": title,
                "url": url
            })
        
        response_dict = {
            "answer": result,
            "references": doc_references
        }

        return response_dict


### **Instâncias dos Prompts**

```python
document_prompt_template = PromptTemplate.from_template("Documento {doc_id}: {page_content}")
map_prompt_template = PromptTemplate.from_template("Resuma o seguinte conteúdo: {context}")
reduce_prompt_template = PromptTemplate.from_template("Combine os seguintes resumos de forma coesa: {context}")


### **Utilização do MapReduceRetriever**

```python
retriever_with_references = MapReduceRetrieverWithReferences(
    llm=llm,
    retriever=retriever,
    document_prompt_template=document_prompt_template,
    map_prompt_template=map_prompt_template,
    reduce_prompt_template=reduce_prompt_template
)

user_input = "Quais são as principais cidades do Japão?"

# Recupera e combina a resposta
result, docs = retriever_with_references.retrieve_and_combine(user_input)

# Formata a resposta final com referências
response = retriever_with_references.format_response_with_references(result, docs)

print(response)

```json
{
    "answer": "As principais cidades do Japão incluem Tóquio, Kyoto e Osaka.",
    "references": [
        {
            "doc_id": "doc1",
            "title": "Cidades Japonesas",
            "url": "http://example.com/doc1"
        },
        {
            "doc_id": "doc2",
            "title": "História do Japão",
            "url": "http://example.com/doc2"
        }
    ]
}
